# Project 2 Code
- all final code that goes into the report will be pulled from here

**To Do:**
1. The prices are still a little mysterious. When I convert the currency to usd, sometimes the number looks right, but sometimes it doesn't... We could either make an assumption that everything is in USD (which is prob unlikely) or we could try to dig a little deeper
2. I broadly categorized the amenities to be tv	wifi	chromecast	refrigerator	kitchen	view	parking	workspace. Please feel free to add more!
3. I changed the "New" rating to be -1 so that we could do some math manipulation later. I think we need to make an assumption later on what the placeholder is going to be for those listings.
4. If you guys find more thing that we need to do, then please add them to the summary table below on "Summary of things To Do and fix"

### Import python packages and data

In [40]:
import pandas as pd
import numpy as np
import pprint


In [41]:
airbnb = pd.read_csv('airbnb.csv')
country_to_letters = pd.read_csv('country-code-to-currency-code-mapping.csv')
pd.set_option('display.max_columns', None)

### Data cleaning

#### Summary of things To Do and fix
| Column | Number of non null values | Original Type | Type to Change to | To Do |
| ------ | ------------------------- | ------------- | ----------------- | ----- |
| Unnamed: 0    | 12805 |  int64   | | DROP |
| id            | 12805 |  int64   | |  |
| name          | 12805 |  object  | str | DROP after matching id and name |
| rating        | 12805 |  object  | str | * mainly float values, but when there are no rating, says "new" </br> * rating does not go below 3.0|
| reviews       | 12805 |  object  | int | * change type to int to make reviews easy to work with |
| host_name     | 12797 |  object  | | DROP after matching host name and host id |
| host_id       | 12805 |  float64 | int | |
| address       | 12805 |  object  | |  |
| features      | 12805 |  object  | | DROP since we have a summary of the features under other columns (toiles, bedrooms, studios, checkin, checkout) |
| amenities     | 12805 |  object  | | values are strung together with commas (,). when analyzing, we need to parse the strings |
| safety_rules  | 12805 |  object  | | values are strung together with commas (,). when analyzing, we need to parse the strings |
| hourse_rules  | 12805 |  object  | | * values are strung together with commas (,). when analyzing, we need to parse the strings </br> * rename column from hourse_rul to house_rule |
| img_links     | 12805 |  object  | | potentially use this column to see if the images help with reviews, but last in priority|
| price         | 12805 |  int64   | float | * Need to find out if the prices are recorded as US dollars  </br> * currently recroded as int, but investigate how it's being recorded|
| country       | 12805 |  object  | str | * Values have whitespaces, should remove |
| bathrooms     | 12805 |  int64   | | |
| beds          | 12805 |  int64   | | |
| guests        | 12805 |  int64   | | |
| toiles        | 12805 |  int64   | | * need to rename column|
| bedrooms      | 12805 |  int64   | | |
| studios       | 12805 |  int64   | | |
| checkin       | 12005 |  object  | | |
| checkout      | 10355 |  object  | datetime | * change to datetime </br> * figure out what to do with the null values|

In [42]:
# 1. Rename all the columns that are mispelled
column_rename = {
  'hourse_rules': 'house_rules',
  'toiles': 'toilets'
}
airbnb = airbnb.rename(columns=column_rename)
airbnb.head()

,Unnamed: 0,id,name,rating,reviews,host_name,host_id,address,features,amenities,safety_rules,house_rules,img_links,price,country,bathrooms,beds,guests,toilets,bedrooms,studios,checkin,checkout
0,0,49849504,Perla bungalov,4.71,64,Mehmetcan,357334205.0,"Kartepe, Kocaeli, Turkey","2 guests,2 bedrooms,1 bed,1 bathroom","Mountain view,Valley view,Lake access,Kitchen,...","󹀁,Airbnb's COVID-19 safety practices apply,󱠃,N...","Check-in: Flexible,Check out: 12:00 pm,Pets ar...",https://a0.muscache.com/im/pictures/a5da5cb7-c...,8078,Turkey,1,1,2,0,2,0,Flexible,12 00 pm
1,1,50891766,Authentic Beach Architect Sheltered Villa with...,New,0,Fatih,386223873.0,"Kaş, Antalya, Turkey","4 guests,2 bedrooms,2 beds,2 bathrooms","Kitchen,Wifi,Dedicated workspace,Free parking ...","󹀁,Airbnb's COVID-19 safety practices apply,󱠆,C...","Check-in: 4:00 pm - 11:00 pm,Check out: 10:00 ...",https://a0.muscache.com/im/pictures/61b70855-2...,4665,Turkey,2,2,4,0,2,0,4 00 pm - 11 00 pm,10 00 am
2,2,50699164,cottages sataplia,4.85,68,Giorgi,409690853.0,"Imereti, Georgia","4 guests,1 bedroom,3 beds,1 bathroom","Mountain view,Kitchen,Wifi,Dedicated workspace...","󹀁,Airbnb's COVID-19 safety practices apply,󱠃,N...","Check-in: After 1:00 pm,Check out: 12:00 pm,Se...",https://a0.muscache.com/im/pictures/miso/Hosti...,5991,Georgia,1,3,4,0,1,0,After 1 00 pm,12 00 pm
3,3,49871422,Sapanca Breathable Bungalow,5.0,13,Melih,401873242.0,"Sapanca, Sakarya, Turkey","4 guests,1 bedroom,2 beds,1 bathroom","Mountain view,Valley view,Kitchen,Wifi,Free pa...","󹀁,Airbnb's COVID-19 safety practices apply,󱠃,N...","Check-in: After 2:00 pm,Check out: 12:00 pm,No...",https://a0.muscache.com/im/pictures/72e6396e-e...,11339,Turkey,1,2,4,0,1,0,After 2 00 pm,12 00 pm
4,4,51245886,Bungalov Ev 2,New,0,Arp Sapanca,414884116.0,"Sapanca, Sakarya, Turkey","2 guests,1 bedroom,1 bed,1 bathroom","Kitchen,Wifi,Free parking on premises,TV,Air c...","󹀁,Airbnb's COVID-19 safety practices apply,󱠆,C...","Check-in: After 2:00 pm,Check out: 12:00 pm,No...",https://a0.muscache.com/im/pictures/73973308-e...,6673,Turkey,1,1,2,0,1,0,After 2 00 pm,12 00 pm


In [43]:
# 2. Change all datatype to its appropriate type

print("Before changing the types of column")
# replace all commas on "reviews" column
airbnb.reviews.replace({",":""}, inplace=True, regex=True)
# replace all New ratings as -1 to convert column to floats
airbnb.rating.replace({"New":"-1"}, inplace=True, regex=True)

print(airbnb.dtypes)

col_dtypes = {
  'name': 'string',
  'rating': 'float64',
  'reviews': 'int64',
  'host_id': 'int64',
  'price': 'float64',
  'country': 'string',
  'checkin': 'string'
}
airbnb = airbnb.astype(col_dtypes, errors = 'ignore')
# airbnb['checkout'] = pd.to_datetime(airbnb['checkout'], errors = 'coerce')
print("\nAfter changing the types of column")
airbnb.dtypes
# airbnb

Before changing the types of column
Unnamed: 0        int64
id                int64
name             object
rating           object
reviews          object
host_name        object
host_id         float64
address          object
features         object
amenities        object
safety_rules     object
house_rules      object
img_links        object
price             int64
country          object
bathrooms         int64
beds              int64
guests            int64
toilets           int64
bedrooms          int64
studios           int64
checkin          object
checkout         object
dtype: object

After changing the types of column


Unnamed: 0        int64
id                int64
name             string
rating          float64
reviews           int64
host_name        object
host_id           int64
address          object
features         object
amenities        object
safety_rules     object
house_rules      object
img_links        object
price           float64
country          string
bathrooms         int64
beds              int64
guests            int64
toilets           int64
bedrooms          int64
studios           int64
checkin          string
checkout         object
dtype: object

#### Assumption 1: The Prices are in country's currency
3. Use 3rd party api to convert the money
* pycountry and currentcurrency package to find out foreign currency to usd


In [44]:
# redefine country to match all.csv country name

# Unique country names from all_countries
all_countries_set = set(airbnb['country'].str.strip())
# Unique country names from country_to_letters
country_to_letters_set = set(country_to_letters['Country'])
missing_countries = all_countries_set - country_to_letters_set
pprint.pp(missing_countries)

# # change the values of country to match the all.csv file
country_change = {
  'Australia': 'Australian',
  'Bosnia & Herzegovina': 'Bosnia and Herzegovina',
  'Brunei': 'Brunei Darussalam',
  'Congo': 'Congo Republic of the Democratic',
  'Croatia': 'Croatia (Hrvatska)',
  'Czechia': 'Czech Republic',
  # kosovo could be equivalent to germany bc they both use euro
  'Kosovo': 'Germany',
  'Slovakia': 'Slovakia (Slovak Republic)',
  'South Korea': 'Korea South',
  'St Martin': 'Saint Martin (French part)',
  'Svalbard & Jan Mayen': 'Svalbard and Jan Mayen Islands',
  # Åland Islands could be equivalent to germany bc they both use euro
  'Åland Islands': 'Germany'}

airbnb.country.replace(country_change, inplace=True, regex=True)

{'Australia',
 'Bosnia & Herzegovina',
 'Brunei',
 'Congo',
 'Croatia',
 'Czechia',
 'Kosovo',
 'Slovakia',
 'South Korea',
 'St Martin',
 'Svalbard & Jan Mayen',
 'Åland Islands'}


In [45]:
# Function to convert price to USD
def convert_price_to_usd(row, converter):
    try:
        # remove whitespace
        country_clean = row['country'].strip()
        # locate the country on country code to currency code df and find 3 digit currency code
        country_code_series = country_to_letters.loc[country_to_letters['Country'] == country_clean]['Code']
        # should have one currency code
        country_code = country_code_series.values[0]
        # change from country code price to usd
        usd_price = converter.convert(row['price'], country_code, 'USD')
        return usd_price
    # if any error happens print and return none
    except Exception as e:
        print(f"Error converting price for {row['country']}: {e}")
        return None

# Initialize the converter
converter = CurrencyConverter()

# Apply the conversion function to the DataFrame
airbnb['usd_price'] = airbnb.apply(convert_price_to_usd, axis=1, converter=converter)

NameError: name 'CurrencyConverter' is not defined

In [32]:
airbnb.head()

,Unnamed: 0,id,name,rating,reviews,host_name,host_id,address,features,amenities,safety_rules,house_rules,img_links,price,country,bathrooms,beds,guests,toilets,bedrooms,studios,checkin,checkout
0,0,49849504,Perla bungalov,4.71,64,Mehmetcan,357334205,"Kartepe, Kocaeli, Turkey","2 guests,2 bedrooms,1 bed,1 bathroom","Mountain view,Valley view,Lake access,Kitchen,...","󹀁,Airbnb's COVID-19 safety practices apply,󱠃,N...","Check-in: Flexible,Check out: 12:00 pm,Pets ar...",https://a0.muscache.com/im/pictures/a5da5cb7-c...,8078.0,Turkey,1,1,2,0,2,0,Flexible,12 00 pm
1,1,50891766,Authentic Beach Architect Sheltered Villa with...,-1.00,0,Fatih,386223873,"Kaş, Antalya, Turkey","4 guests,2 bedrooms,2 beds,2 bathrooms","Kitchen,Wifi,Dedicated workspace,Free parking ...","󹀁,Airbnb's COVID-19 safety practices apply,󱠆,C...","Check-in: 4:00 pm - 11:00 pm,Check out: 10:00 ...",https://a0.muscache.com/im/pictures/61b70855-2...,4665.0,Turkey,2,2,4,0,2,0,4 00 pm - 11 00 pm,10 00 am
2,2,50699164,cottages sataplia,4.85,68,Giorgi,409690853,"Imereti, Georgia","4 guests,1 bedroom,3 beds,1 bathroom","Mountain view,Kitchen,Wifi,Dedicated workspace...","󹀁,Airbnb's COVID-19 safety practices apply,󱠃,N...","Check-in: After 1:00 pm,Check out: 12:00 pm,Se...",https://a0.muscache.com/im/pictures/miso/Hosti...,5991.0,Georgia,1,3,4,0,1,0,After 1 00 pm,12 00 pm
3,3,49871422,Sapanca Breathable Bungalow,5.00,13,Melih,401873242,"Sapanca, Sakarya, Turkey","4 guests,1 bedroom,2 beds,1 bathroom","Mountain view,Valley view,Kitchen,Wifi,Free pa...","󹀁,Airbnb's COVID-19 safety practices apply,󱠃,N...","Check-in: After 2:00 pm,Check out: 12:00 pm,No...",https://a0.muscache.com/im/pictures/72e6396e-e...,11339.0,Turkey,1,2,4,0,1,0,After 2 00 pm,12 00 pm
4,4,51245886,Bungalov Ev 2,-1.00,0,Arp Sapanca,414884116,"Sapanca, Sakarya, Turkey","2 guests,1 bedroom,1 bed,1 bathroom","Kitchen,Wifi,Free parking on premises,TV,Air c...","󹀁,Airbnb's COVID-19 safety practices apply,󱠆,C...","Check-in: After 2:00 pm,Check out: 12:00 pm,No...",https://a0.muscache.com/im/pictures/73973308-e...,6673.0,Turkey,1,1,2,0,1,0,After 2 00 pm,12 00 pm


No conclusion on the price column. Can't tell whehter it's because of wrong decimal places or in foreign currency.
I converted from foreign currency to US dollars, but USD looks correct for some but not all. I think we need to investigate this column a little bit more...

jon: thanks Sarah! Ravi and I managed to find some listings and the prices seem quite similar to those in the local currency... I think we can probably just caveat that it's in local currency? What are some examples of rows that didn't seem right in USD?

In [52]:
# create a sorted list of unique amenities so we can see what are the key amenities we want to look at
unique_amenities = sorted(list(airbnb['amenities'].str.split(',').explode().unique()))
unique_amenities

[' 2–5 years old',
 ' 2–5 years old and 5–10 years old',
 ' 5–10 years old and 10+ years old',
 ' Amazon Prime Video',
 ' Apple TV',
 ' Chromecast',
 ' DVD player',
 ' Disney+',
 ' Fire TV',
 ' HBO Max',
 ' Hulu',
 ' Netflix',
 ' Roku',
 ' cable/satellite TV',
 ' gas powered fridge. refrigerator',
 ' heated',
 ' infinity',
 ' lap pool',
 ' olympic-sized',
 ' open 24 hours',
 ' open specific hours',
 ' rooftop',
 ' saltwater',
 ' standard cable/satellite',
 ' sun loungers',
 ' wood-burning',
 '100" HDTV with Amazon Prime Video',
 '100" TV',
 '100" TV with Chromecast',
 '100" TV with Netflix',
 '101" HDTV with HBO Max',
 '107" HDTV with standard cable/satellite',
 '108" HDTV with standard cable/satellite',
 '12" TV with Amazon Prime Video',
 '12" TV with Netflix',
 '12" TV with standard cable/satellite',
 '12v cool box refrigerator',
 '14" HDTV with cable/satellite TV',
 '14" TV',
 '15" TV',
 '15" TV with standard cable/satellite',
 '16" TV',
 '165" HDTV with cable/satellite TV',
 '17" T

In [54]:
# First, we want to make a nother column that holds the lower case list of all ammenities
airbnb['amenities_list'] = airbnb['amenities'].apply(lambda x: [item.lower().strip() for item in x.split(',')])

# I'm adding keywords that could help us find the 'common amenities'
# each of these keys will be another column and the value will be either yes or no (depending if listing contains these keywords)
# please feel free to add more amenities categories
# we could also do the same thing for safety rules if we want to analyze that as well
categories = {
    'tv': ['tv'],
    'wifi': ['wifi'],
    'chromecast': ['chromecast'],
    'refrigerator': ['refrigerator', 'fridge'],
    'kitchen': ['kitchen'],
    'view': ['view'],
    'parking': ['parking'],
    'workspace': ['workspace'],
    'pool': ['pool'],
    'gym': ['gym'],    
}

# Initialize columns with "no"
for category in categories:
    airbnb[category] = "no"

# Update columns with "yes" where the amenity is present
for category, keywords in categories.items():
    airbnb[category] = airbnb['amenities_list'].apply(
        lambda x: "yes" if any(keyword in ' '.join(x) for keyword in keywords) else "no"
    )

airbnb.head()

,Unnamed: 0,id,name,rating,reviews,host_name,host_id,address,features,amenities,safety_rules,house_rules,img_links,price,country,bathrooms,beds,guests,toilets,bedrooms,studios,checkin,checkout,amenities_list,tv,wifi,chromecast,refrigerator,kitchen,view,parking,workspace,pool,gym
0,0,49849504,Perla bungalov,4.71,64,Mehmetcan,357334205,"Kartepe, Kocaeli, Turkey","2 guests,2 bedrooms,1 bed,1 bathroom","Mountain view,Valley view,Lake access,Kitchen,...","󹀁,Airbnb's COVID-19 safety practices apply,󱠃,N...","Check-in: Flexible,Check out: 12:00 pm,Pets ar...",https://a0.muscache.com/im/pictures/a5da5cb7-c...,8078.0,Turkey,1,1,2,0,2,0,Flexible,12 00 pm,"[mountain view, valley view, lake access, kitc...",yes,yes,no,no,yes,yes,yes,no,no,no
1,1,50891766,Authentic Beach Architect Sheltered Villa with...,-1.00,0,Fatih,386223873,"Kaş, Antalya, Turkey","4 guests,2 bedrooms,2 beds,2 bathrooms","Kitchen,Wifi,Dedicated workspace,Free parking ...","󹀁,Airbnb's COVID-19 safety practices apply,󱠆,C...","Check-in: 4:00 pm - 11:00 pm,Check out: 10:00 ...",https://a0.muscache.com/im/pictures/61b70855-2...,4665.0,Turkey,2,2,4,0,2,0,4 00 pm - 11 00 pm,10 00 am,"[kitchen, wifi, dedicated workspace, free park...",yes,yes,no,no,yes,no,yes,yes,yes,no
2,2,50699164,cottages sataplia,4.85,68,Giorgi,409690853,"Imereti, Georgia","4 guests,1 bedroom,3 beds,1 bathroom","Mountain view,Kitchen,Wifi,Dedicated workspace...","󹀁,Airbnb's COVID-19 safety practices apply,󱠃,N...","Check-in: After 1:00 pm,Check out: 12:00 pm,Se...",https://a0.muscache.com/im/pictures/miso/Hosti...,5991.0,Georgia,1,3,4,0,1,0,After 1 00 pm,12 00 pm,"[mountain view, kitchen, wifi, dedicated works...",yes,yes,no,no,yes,yes,yes,yes,no,no
3,3,49871422,Sapanca Breathable Bungalow,5.00,13,Melih,401873242,"Sapanca, Sakarya, Turkey","4 guests,1 bedroom,2 beds,1 bathroom","Mountain view,Valley view,Kitchen,Wifi,Free pa...","󹀁,Airbnb's COVID-19 safety practices apply,󱠃,N...","Check-in: After 2:00 pm,Check out: 12:00 pm,No...",https://a0.muscache.com/im/pictures/72e6396e-e...,11339.0,Turkey,1,2,4,0,1,0,After 2 00 pm,12 00 pm,"[mountain view, valley view, kitchen, wifi, fr...",yes,yes,no,no,yes,yes,yes,no,yes,no
4,4,51245886,Bungalov Ev 2,-1.00,0,Arp Sapanca,414884116,"Sapanca, Sakarya, Turkey","2 guests,1 bedroom,1 bed,1 bathroom","Kitchen,Wifi,Free parking on premises,TV,Air c...","󹀁,Airbnb's COVID-19 safety practices apply,󱠆,C...","Check-in: After 2:00 pm,Check out: 12:00 pm,No...",https://a0.muscache.com/im/pictures/73973308-e...,6673.0,Turkey,1,1,2,0,1,0,After 2 00 pm,12 00 pm,"[kitchen, wifi, free parking on premises, tv, ...",yes,yes,no,no,yes,no,yes,no,no,no


#### Assumption 2: When there are no checkout time, then we will assume it's flexible

In [36]:
airbnb["checkout"].fillna("Flexible", inplace = True) 

Drop duplicates based on airbnb name and host id

In [37]:
airbnb[airbnb.sort_values(by = ['host_id']).duplicated(subset=['host_id', 'name'], keep = False)].head(50)

/var/folders/wj/66hs1jcj3cx6mtfz_7g_6hsr0000gn/T/ipykernel_10642/4132225726.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  airbnb[airbnb.sort_values(by = ['host_id']).duplicated(subset=['host_id', 'name'], keep = False)].head(50)


,Unnamed: 0,id,name,rating,reviews,host_name,host_id,address,features,amenities,safety_rules,house_rules,img_links,price,country,bathrooms,beds,guests,toilets,bedrooms,studios,checkin,checkout,amenities_list,tv,wifi,chromecast,refrigerator,kitchen,view,parking,workspace
85,85,593801499660969808,Nirvana | A Frame Cabin,4.75,4,Sukhbir,425671475,"Jibhi, Himachal Pradesh, India","2 guests,1 bedroom,1 bed,1 bathroom","Mountain view,Kitchen,Wifi,Dedicated workspace...","󹀁,Airbnb's COVID-19 safety practices apply,󱠆,C...","Check-in: 11:00 am - 2:00 pm,Check out: 11:00 ...",https://a0.muscache.com/im/pictures/miso/Hosti...,5135.0,India,1,1,2,0,1,0,11 00 am - 2 00 pm,11 00 am,"[mountain view, kitchen, wifi, dedicated works...",no,yes,no,no,yes,yes,yes,yes
122,122,547489956860150961,BUNGALOW VİLLA WITH NATURE AND LAKE VIEW,-1.00,0,Fatih,386223873,"Sapanca, Sakarya, Turkey","2 guests,1 bedroom,1 bed,1 bathroom","Garden view,Lake view,Kitchen,Wifi,Dedicated w...","󹀁,Airbnb's COVID-19 safety practices apply,󱠆,C...","Check-in: 2:00 pm - 12:00 am,Check out: 12:00 ...",https://a0.muscache.com/im/pictures/miso/Hosti...,41682.0,Turkey,1,1,2,0,1,0,2 00 pm - 12 00 am,12 00 pm,"[garden view, lake view, kitchen, wifi, dedica...",yes,yes,no,no,yes,yes,yes,yes
124,124,31006192,"(,)",4.67,3,政憲,22474324,"Zuozhen District, Tainan City, Taiwan","2 guests,1 bedroom,2 beds,1 shared bathroom","Wifi,Free parking on premises,Air conditioning...","󹀁,Airbnb's COVID-19 safety practices apply,󱠃,N...","Check-in: After 2:00 pm,No smoking,No pets,Sho...",https://a0.muscache.com/im/pictures/f15983d3-f...,5559.0,Taiwan,1,2,2,0,1,0,After 2 00 pm,Flexible,"[wifi, free parking on premises, air condition...",no,yes,no,no,no,no,yes,no
127,127,52763783,Nirvana | A Frame Cabin,5.00,3,Sukhbir,425671475,"Jibhi, Himachal Pradesh, India","2 guests,1 bedroom,1 bed,1 private bathroom","Valley view,Mountain view,Kitchen,Wifi,Dedicat...","󹀁,Airbnb's COVID-19 safety practices apply,󱠆,C...","Check-in: 11:00 am - 2:00 pm,Check out: 11:00 ...",https://a0.muscache.com/im/pictures/miso/Hosti...,5135.0,India,1,1,2,0,1,0,11 00 am - 2 00 pm,11 00 am,"[valley view, mountain view, kitchen, wifi, de...",no,yes,no,no,yes,yes,yes,yes
136,136,547548961987730165,BUNGALOW VİLLA WITH NATURE AND LAKE VIEW,-1.00,0,Fatih,386223873,"Sapanca, Sakarya, Turkey","2 guests,1 bedroom,1 bed,1 bathroom","Garden view,Lake view,Kitchen,Wifi,Dedicated w...","󹀁,Airbnb's COVID-19 safety practices apply,󱠆,C...","Check-in: 2:00 pm - 12:00 am,Check out: 12:00 ...",https://a0.muscache.com/im/pictures/miso/Hosti...,44109.0,Turkey,1,1,2,0,1,0,2 00 pm - 12 00 am,12 00 pm,"[garden view, lake view, kitchen, wifi, dedica...",yes,yes,no,no,yes,yes,yes,yes
138,138,30916398,"(,)",5.00,3,政憲,22474324,"Zuozhen District, Tainan City, Taiwan","2 guests,1 bedroom,2 beds,1 shared bathroom","Wifi,Free parking on premises,Air conditioning...","󹀁,Airbnb's COVID-19 safety practices apply,󱠃,N...","Check-in: After 2:00 pm,No smoking,No pets,Sho...",https://a0.muscache.com/im/pictures/f15983d3-f...,5559.0,Taiwan,1,2,2,0,1,0,After 2 00 pm,Flexible,"[wifi, free parking on premises, air condition...",no,yes,no,no,no,no,yes,no
724,724,576430996210237176,Apartment facing breathtaking Victoria garden ...,5.00,9,Juzer,81327040,"Mumbai, Maharashtra, India","3 guests,1 bedroom,1 bed,1 private bathroom","City skyline view,Garden view,Kitchen,Wifi – 4...","󹀁,Airbnb's COVID-19 safety practices apply,󱠆,C...","Check-in: 12:00 pm - 5:00 pm,Check out: 10:00 ...",https://a0.muscache.com/im/pictures/990ae6e3-a...,1963.0,India,1,1,3,0,1,0,12 00 pm - 5 00 pm,10 00 am,"[city skyline view, garden view, kitchen, wifi...",no,yes,no,yes,yes,yes,yes,no
749,749,619404025118853400,Apartment facing breathtaking Victoria garden ...,4.86,7,Juzer,81327040,"Mumbai, Maharashtra, India","3 guests,1 bedroom,1 bed,1 private bathroom","Garden view,Sea view,Kitchen,Wifi – 48 Mbps,Fr...","󹀁,Airbnb's COVID-19 safety practices apply,󱠆,C...","Check-in: 12:00 pm - 5:00 pm,Check out: 10:0

In [38]:
airbnb.drop_duplicates(subset=['name', 'host_id'], keep='first')

,Unnamed: 0,id,name,rating,reviews,host_name,host_id,address,features,amenities,safety_rules,house_rules,img_links,price,country,bathrooms,beds,guests,toilets,bedrooms,studios,checkin,checkout,amenities_list,tv,wifi,chromecast,refrigerator,kitchen,view,parking,workspace
0,0,49849504,Perla bungalov,4.71,64,Mehmetcan,357334205,"Kartepe, Kocaeli, Turkey","2 guests,2 bedrooms,1 bed,1 bathroom","Mountain view,Valley view,Lake access,Kitchen,...","󹀁,Airbnb's COVID-19 safety practices apply,󱠃,N...","Check-in: Flexible,Check out: 12:00 pm,Pets ar...",https://a0.muscache.com/im/pictures/a5da5cb7-c...,8078.0,Turkey,1,1,2,0,2,0,Flexible,12 00 pm,"[mountain view, valley view, lake access, kitc...",yes,yes,no,no,yes,yes,yes,no
1,1,50891766,Authentic Beach Architect Sheltered Villa with...,-1.00,0,Fatih,386223873,"Kaş, Antalya, Turkey","4 guests,2 bedrooms,2 beds,2 bathrooms","Kitchen,Wifi,Dedicated workspace,Free parking ...","󹀁,Airbnb's COVID-19 safety practices apply,󱠆,C...","Check-in: 4:00 pm - 11:00 pm,Check out: 10:00 ...",https://a0.muscache.com/im/pictures/61b70855-2...,4665.0,Turkey,2,2,4,0,2,0,4 00 pm - 11 00 pm,10 00 am,"[kitchen, wifi, dedicated workspace, free park...",yes,yes,no,no,yes,no,yes,yes
2,2,50699164,cottages sataplia,4.85,68,Giorgi,409690853,"Imereti, Georgia","4 guests,1 bedroom,3 beds,1 bathroom","Mountain view,Kitchen,Wifi,Dedicated workspace...","󹀁,Airbnb's COVID-19 safety practices apply,󱠃,N...","Check-in: After 1:00 pm,Check out: 12:00 pm,Se...",https://a0.muscache.com/im/pictures/miso/Hosti...,5991.0,Georgia,1,3,4,0,1,0,After 1 00 pm,12 00 pm,"[mountain view, kitchen, wifi, dedicated works...",yes,yes,no,no,yes,yes,yes,yes
3,3,49871422,Sapanca Breathable Bungalow,5.00,13,Melih,401873242,"Sapanca, Sakarya, Turkey","4 guests,1 bedroom,2 beds,1 bathroom","Mountain view,Valley view,Kitchen,Wifi,Free pa...","󹀁,Airbnb's COVID-19 safety practices apply,󱠃,N...","Check-in: After 2:00 pm,Check out: 12:00 pm,No...",https://a0.muscache.com/im/pictures/72e6396e-e...,11339.0,Turkey,1,2,4,0,1,0,After 2 00 pm,12 00 pm,"[mountain view, valley view, kitchen, wifi, fr...",yes,yes,no,no,yes,yes,yes,no
4,4,51245886,Bungalov Ev 2,-1.00,0,Arp Sapanca,414884116,"Sapanca, Sakarya, Turkey","2 guests,1 bedroom,1 bed,1 bathroom","Kitchen,Wifi,Free parking on premises,TV,Air c...","󹀁,Airbnb's COVID-19 safety practices apply,󱠆,C...","Check-in: After 2:00 pm,Check out: 12:00 pm,No...",https://a0.muscache.com/im/pictures/73973308-e...,6673.0,Turkey,1,1,2,0,1,0,After 2 00 pm,12 00 pm,"[kitchen, wifi, free parking on premises, tv, ...",yes,yes,no,no,yes,no,yes,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12800,12800,44697740,"Brown Dome House No. 3, full of healing",4.94,16,Aidan,161552316,"Imhoe-myeon, Jindo, South Jeolla Province, Sou...","4 guests,8 bathrooms","Kitchen,Wifi,Free parking on premises,Private ...","󹀁,Airbnb's COVID-19 safety practices apply,󱠁,C...","Check-in: 3:00 pm - 11:00 pm,Check out: 11:00 ...",https://a0.muscache.com/im/pictures/miso/Hosti...,4288.0,Korea South,8,0,4,0,0,0,3 00 pm - 11 00 pm,11 00 am,"[kitchen, wifi, free parking on premises, priv...",yes,yes,no,no,yes,no,yes,no
12801,12801,29975283,Bubble Hotel Ubud with Bathtub under the Sky,4.67,6,Martha,118954227,"Gianyar, Indonesia","2 guests,1 bedroom,1 bed,Toilet with sink","Wifi,Free on-street parking,Air conditioning,G...","󹀁,Airbnb's COVID-19 safety practices apply,󱠆,C...","Check-in: 3:00 pm - 10:00 pm,Check out: 12:00 ...",https://a0.muscache.com/im/pictures/5ba944b2-b...,15200.0,Indonesia,0,1,2,1,1,0,3 00 pm - 10 00 pm,12 00 pm,"[wifi, free on-street parking, air conditionin...",no,yes,no,no,no,no,yes,no
12802,12802,32977697,Paradise Eco Hub,4.67,3,Silas,175214568,"Kabale, Uganda","2 guests,1 bedroom,2 beds,1 private bathroom","Wifi,Breakfast,Long-term stays allowed,Unavail...","󹀁,Airbnb's COVID-19 safety practices apply,󱠃,N...",Check-in: After 3:00 pm,https://a0.muscach

In [39]:
# Drop unnecessary columns Unnamed: 0, host_name,	address, features,	amenities, amenities_list, house_rules, price (because we have new column called usd_price)
airbnb.drop(['Unnamed: 0', 'id', 'host_name',	'address', 'features',	'amenities', 'amenities_list', 'house_rules', 'safety_rules', 'price'], axis=1, inplace=True)